In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [6]:
dataset.drop("User ID",axis=1)

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [7]:
dataset["Purchased"].value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [8]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]
independent

,Age,EstimatedSalary,Gender_Male
0,19,19000,1
1,35,20000,1
2,26,43000,0
3,27,57000,0
4,19,76000,1
...,...,...,...
395,46,41000,0
396,51,23000,1
397,50,20000,0
398,36,33000,1


In [9]:
dependent=dataset[['Purchased']]
dependent

,Purchased
0,0
1,0
2,0
3,0
4,0
...,...
395,1
396,1
397,1
398,0


In [10]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=1/3,random_state=0)

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
param_grid={'criterion':['gini','entropy'],
            'max_features':['auto','sqrt','log2'],
            'n_estimators':[10,100]           
    
            }
grid=GridSearchCV(RandomForestClassifier(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1')
grid.fit(X_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Users\yuvar\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


GridSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1', verbose=3)

In [13]:
grid_pred=grid.predict(X_test)

In [14]:
result=grid.cv_results_

In [15]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_pred)
print(cm)

[[78  7]
 [ 5 44]]


In [16]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.94      0.92      0.93        85
           1       0.86      0.90      0.88        49

    accuracy                           0.91       134
   macro avg       0.90      0.91      0.90       134
weighted avg       0.91      0.91      0.91       134



In [21]:
table=pd.DataFrame.from_dict(result)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_features,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.039573,0.009868,0.010867,0.009478,gini,auto,10,"{'criterion': 'gini', 'max_features': 'auto', ...",0.764706,0.764706,0.809524,0.900000,0.947368,0.837261,0.073970,3
1,0.237592,0.016770,0.027421,0.004502,gini,auto,100,"{'criterion': 'gini', 'max_features': 'auto', ...",0.727273,0.777778,0.780488,0.900000,0.923077,0.821723,0.076095,7
2,0.041913,0.012411,0.008563,0.005420,gini,sqrt,10,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.687500,0.756757,0.731707,0.871795,0.918919,0.793336,0.087481,11
3,0.245116,0.022116,0.029828,0.005518,gini,sqrt,100,"{'criterion': 'gini', 'max_features': 'sqrt', ...",0.727273,0.842105,0.780488,0.900000,0.923077,0.834589,0.072951,5
4,0.044596,0.007255,0.009494,0.005904,gini,log2,10,"{'criterion': 'gini', 'max_features': 'log2', ...",0.800000,0.769231,0.731707,0.900000,0.857143,0.811616,0.060335,9
5,0.250080,0.017698,0.029667,0.003899,gini,log2,100,"{'criterion': 'gini', 'max_features': 'log2', ...",0.727273,0.850000,0.780488,0.900000,0.923077,0.836167,0.073181,4
6,0.032502,0.003785,0.012245,0.008376,entropy,auto,10,"{'criterion': 'entropy', 'max_features': 'auto...",0.727273,0.820513,0.780488,0.871795,0.882353,0.816484,0.057738,8
7,0.258367,0.022218,0.032894,0.008833,entropy,auto,100,"{'criterion': 'entropy', 'max_features': 'auto...",0.727273,0.789474,0.809524,0.900000,0.923077,0.829869,0.072357,6
8,0.037416,0.009410,0.010801,0.008859,entropy,sqrt,10,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.666667,0.777778,0.780488,0.871795,0.823529,0.784051,0.067954,12
9,0.237090,0.017593,0.024708,0.007922,entropy,sqrt,100,"{'criterion': 'entropy', 'max_features': 'sqrt...",0.727273,0.871795,0.780488,0.900000,0.947368,0.845385,0.080303,2


In [17]:
from sklearn.metrics import f1_score
f_score=f1_score(y_test,grid_pred,average='weighted')
print("the f1 value for best parameter{}".format(grid.best_params_),f_score)

the f1 value for best parameter{'criterion': 'entropy', 'max_features': 'log2', 'n_estimators': 100} 0.9108102345415777


In [18]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.9702280912364946